# Type II Equilibrium Plots

In [2]:
import numpy as np
import matplotlib.pyplot as plt
final_fig_path = "../Manuscript/Figures/"
import sys
sys.path.insert(1, 'Functions')
from fitness_funs_non_dim import *
from group_w_pop_funs import *
from scipy.integrate import solve_ivp
from sim_graph_funs import *


In [ ]:
def get_equilibrium(params, N1_0 = 0.5, N2_0 = 0.4, P_0 = 5, F_of_x_vec = None):
    '''
    finds the equilibrium using Fsolve
    if not given F_of_x_vec, then just has everyone initially solitary
    
    @returns:
    N1_eq, N2_eq, F_eq, P_eq, mean_x_eq
    '''
    x_max = params['x_max']
    if ~isinstance(F_of_x_vec, np.ndarray):
        F_of_x_vec = np.zeros(x_max)
        F_of_x_vec[0] = P_0
    x0 = [N1_0, N2_0, *F_of_x_vec]
    out, infodict, ier, mesg = fsolve(func = nullclines_no_P, x0 = x0, 
                                  args = (params), full_output = 1)
    F_eq = out[2:]
    P_eq = np.sum(np.arange(1,x_max+1,1)*F_eq); 
    N1_eq = out[0]
    N2_eq = out[1]
    mean_x_eq = mean_group_size_membership(F_eq,x_max,P_eq)

    if np.any( np.round(np.array([N1_eq, N2_eq, *F_eq, P_eq, mean_x_eq]),8) <0):
        return np.nan, np.nan, np.nan, np.nan, np.nan
    else:
        return N1_eq, N2_eq, F_eq, P_eq, mean_x_eq